In [1]:
# pytorch mlp for regression
from numpy import vstack
from numpy import sqrt
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_


In [28]:
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1].astype('float32')
        self.y = df.values[:, -1].astype('float32')
    
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 32)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = Sigmoid()
        # second hidden layer
        self.hidden2 = Linear(32, 16)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()
        # third hidden layer and output
        self.hidden3 = Linear(16, 1)
        xavier_uniform_(self.hidden3.weight)

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X

# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=False)
    test_dl = DataLoader(test, batch_size=32, shuffle=False)
    return train_dl, test_dl

# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = MSELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    return mse

# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [29]:
path = './data/qoe/Client1_2.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(13)
# train the model
train_model(train_dl, model)
# evaluate the model
mse = evaluate_model(test_dl, model)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
# make a single prediction (expect class=1)
#row = [0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98]
row = [34011,50,5699,1035,0,6015,0,30003,30003,0,2903,1544292,4.33]
yhat = predict(row, model)
print('Predicted: %.3f' % yhat)

670 330
MSE: 0.245, RMSE: 0.495
Predicted: 3.838


In [30]:
path2 = './data/qoe/Client2_2.csv'
train_dl2, test_dl2 = prepare_data(path2)
print(len(train_dl2.dataset), len(test_dl2.dataset))
# define the network
model2 = MLP(13)
# train the model
train_model(train_dl2, model2)
# evaluate the model
mse2 = evaluate_model(test_dl2, model2)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
# make a single prediction (expect class=1)
#row = [0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98]
row2 = [34011,50,5699,1035,0,6015,0,30003,30003,0,2903,1544292,4.33]
yhat2 = predict(row2, model2)
print('Predicted: %.3f' % yhat)

670 330
MSE: 0.245, RMSE: 0.495
Predicted: 3.838


In [31]:
import torch 
torch.save(model.state_dict(), './model/model_client1.pt')

In [32]:
import torch 
torch.save(model.state_dict(), './model/model_client2.pt')

In [33]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import logging
from pandas import read_csv

# import Pysyft to help us to simulate federated leraning
import syft as sy

# hook PyTorch to PySyft i.e. add extra functionalities to support Federated Learning
# and other private AI tools
hook = sy.TorchHook(torch) 

In [34]:
bob = sy.VirtualWorker(hook, id="bob")
anne = sy.VirtualWorker(hook, id="anne")

In [35]:
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_
from torch import Tensor
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = Sigmoid()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X


In [36]:
import pandas as pd
import torch

# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [37]:
path = './data/qoe/Client1_2.csv'
df = read_csv(path, header=None)
        # store the inputs and outputs
    
train_set=df.sample(frac=0.8,random_state=200) #random state is a seed value
test_set=df.drop(train_set.index)

train_set_data  = train_set.iloc[:, :-1]
target_set_data = train_set.iloc[:,13]

train_tensor = torch.from_numpy(train_set_data.to_numpy()).float()
target_tensor  = torch.from_numpy(target_set_data.to_numpy()).float()

test_tensor = torch.from_numpy(test_set.to_numpy()).float()


In [38]:
path2 = './data/qoe/Client2_2.csv'
df2 = read_csv(path2, header=None)


train_set_2=df2.sample(frac=0.8,random_state=200) #random state is a seed value
test_set_2=df2.drop(train_set.index)

train_set_data2  = train_set_2.iloc[:, :-1]
target_set_data2 = train_set_2.iloc[:,13]



train_tensor_2 = df_to_tensor(train_set_data2)
target_tensor_2 =  df_to_tensor(target_set_data2)
test_tensor_2 = df_to_tensor(test_set_2)

In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import numpy as np

In [40]:
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [41]:
target_tensor = target_tensor.type(torch.LongTensor)
target_tensor_2 = target_tensor_2.type(torch.LongTensor)

bob_train_dataset = sy.BaseDataset(train_tensor,target_tensor).send(bob) 
anne_train_dataset = sy.BaseDataset(train_tensor_2, target_tensor_2).send(alice)

#bob_train_dataset = sy.BaseDataset(train_inputs[:train_idx], train_labels[:train_idx]).send(bob)

#anne_train_dataset = sy.BaseDataset(train_inputs[train_idx:], train_labels[train_idx:]).send(anne)

federated_train_dataset = sy.FederatedDataset([bob_train_dataset, anne_train_dataset]) 
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset, shuffle=False, batch_size=32)

In [47]:
print(federated_train_dataset)

FederatedDataset
    Distributed accross: bob, alice
    Number of datapoints: 1600



In [42]:
model3 = MLP(13).to(device)

In [43]:
print(federated_train_loader)

In [60]:
def train(model, federate_train_loader, optimizer, epoch):
    model.train()
    criterion = MSELoss()
    for batch_idx, (data, target) in enumerate(federate_train_loader):
        model.send(data.location)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target.float())
        loss = criterion(output, target.float())
        loss.backward()
        optimizer.step()
        model.get()
        if batch_idx % 10 == 0:
            loss = loss.get()
            #print(f'Train Epoch: {epoch} [{batch_idx*32}/{len(federate_train_loader)*32} ({100. * batch_idx / len(federated_train_loader)}%)] \t Loss: {loss.item()}')
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, 
                    batch_idx * 32 , # no of images done
                    len(federate_train_loader) * 32, # total images left
                    100. * batch_idx / len(federate_train_loader), 
                    loss.item()
                 )
            )
                  


In [61]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # add losses together
            test_loss += F.mse_loss(output, target.float(), reduction='sum').item() 

            # get the index of the max probability class
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [62]:
optimizer = optim.SGD(model3.parameters(), lr=0.01) 
#optimizer = SGD(model3.parameters(), lr=0.01, momentum=0.9)
    
epoch = 20
for epoch in range(1, epoch + 1):
    train(model3, federated_train_loader, optimizer, epoch)
    #test(model3, test_loader)

#th.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/1600 (0%)]	Loss: 0.346924
Train Epoch: 1 [320/1600 (20%)]	Loss: 0.251998
Train Epoch: 1 [640/1600 (40%)]	Loss: 0.390009
Train Epoch: 1 [960/1600 (60%)]	Loss: 0.444696
Train Epoch: 1 [1280/1600 (80%)]	Loss: 0.440440
Train Epoch: 2 [0/1600 (0%)]	Loss: 0.360719
Train Epoch: 2 [320/1600 (20%)]	Loss: 0.249385
Train Epoch: 2 [640/1600 (40%)]	Loss: 0.390217
Train Epoch: 2 [960/1600 (60%)]	Loss: 0.444326
Train Epoch: 2 [1280/1600 (80%)]	Loss: 0.440334
Train Epoch: 3 [0/1600 (0%)]	Loss: 0.360813
Train Epoch: 3 [320/1600 (20%)]	Loss: 0.249369
Train Epoch: 3 [640/1600 (40%)]	Loss: 0.390219
Train Epoch: 3 [960/1600 (60%)]	Loss: 0.444324
Train Epoch: 3 [1280/1600 (80%)]	Loss: 0.440334
Train Epoch: 4 [0/1600 (0%)]	Loss: 0.360813
Train Epoch: 4 [320/1600 (20%)]	Loss: 0.249369
Train Epoch: 4 [640/1600 (40%)]	Loss: 0.390219
Train Epoch: 4 [960/1600 (60%)]	Loss: 0.444324
Train Epoch: 4 [1280/1600 (80%)]	Loss: 0.440334
Train Epoch: 5 [0/1600 (0%)]	Loss: 0.360813
Train Epoch: 5 [320/1600

In [63]:
import torch 
torch.save(model3.state_dict(), './model/master_model.pt')

In [64]:
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [65]:
#row2 = [34011,50,5699,1035,0,6015,0,30003,30003,0,2903,1544292,4.33]
row =  [57400,57,6023,1051,0,6128,0,30003,30003,0,2903,1565292,4.33]

client = predict(row, model)

In [66]:
print('Predicted: %.3f' % client)

Predicted: 3.838


In [67]:
row2 = [34011,50,5699,1035,0,6015,0,30003,30003,0,2903,1544292,4.33]
#row2 =  [57400,57,6023,1051,0,6128,0,30003,30003,0,2903,1565292,4.33]

master = predict(row2, model3)

In [68]:
print('Predicted: %.3f' % master)

Predicted: 3.360


In [69]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += F.nll_loss(
            output, target, reduction='sum').item()
        # get the index of the max log-probability
        pred = output.argmax(1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss,
    correct,
    len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

NameError: name 'test_loader' is not defined